## Implied Volatility in the Black-Scholes Model

Despite its controversial assumptions, the Black-Scholes model is still widely used in financial markets. In this note, I will discuss **implied volatility**, which is the volatility computed such that the model (Black-Scholes model) price matches the observed market price.

Let $P_m(\sigma, \cdot )$ be the price from the Black-Scholes model and $\sigma$ be the volatility, and let $P_{\text{mkt}}$ be the market price.

If:
$$
P_m(\sigma, \cdot) = P_{\text{mkt}},
$$
then the volatility $\sigma$ that satisfies this equation is called the implied volatility.

In this notebook, I will consider a European option to compute the implied volatility. Let $S$ be the current price of the stock, $K$ be the exercise price, $r$ be the risk free rate, and $T$ be the maturity of the option.

The price of the European call at time zero is given by:

$$P_m(\sigma, S, K, r, T) = S\mathcal{N}(d_1)-K e^{-rT} \mathcal{N}(d_2),$$
where $d_1 = \frac{\log(S/K)+(r+0.5 \sigma^2)T}{\sigma \sqrt{T}}, d_2=d_1-\sigma \sqrt{T}.$

If $P_m(\sigma, \cdot) = P_{\text{mkt}}$, then we need to solve for $\sigma$. We can do this using a root finding technique such as Secant method such that we set function $f$ to be:
$$f(\sigma) = S\mathcal{N}(d_1)-K e^{-rT} \mathcal{N}(d_2)-P_{\text{mkt}}.$$
Newton's method will require us to compute the first derivative $f$ but that's a tedious process, so we cheat this by approximating the first derivative using finite difference methods. Thus we employ Secan't method. Recall Newton's method:
$$\sigma_{n+1}=\sigma_{n}-f(\sigma_{n})/f'(\sigma_n)$$.
We approximate the $f'(\sigma_n)$ by:
$$ f'(\sigma_n) = \frac{f(\sigma_n)-f(\sigma_{n-1})}{\sigma_n-\sigma_{n-1}},$$
Thus, Secant method is:
$$\sigma_{n+1}=\sigma_{n}-f(\sigma_{n})\cdot \frac{\sigma_n-\sigma_{n-1}}{f(\sigma_n)-f(\sigma_{n-1})}$$.
We will need two initial guesses $\sigma_0, \sigma_1$ instead of one like in Newton's method.

## Numerical Examples

We consider a call option with the following parameters: $S = R40, K=35, r=0.08, T=1, P_{mkt}=10$

In [94]:
import numpy as np
from scipy.stats import norm

class SecantMethod:
    def __init__(self, D, S, K, r, T, market_price):
        # D is the interval for which we are searching for the roots
        self.D = D
        self.S = S  # Underlying asset price
        self.K = K  # Strike price
        self.r = r  # Risk-free interest rate
        self.T = T  # Time to maturity
        self.market_price = market_price  # Market price of the option

    def bsm_call_price(self, sigma):
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * sigma**2) * self.T) / (sigma * np.sqrt(self.T))
        d2 = d1 - sigma * np.sqrt(self.T)
        return self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)

    def f(self, sigma): return self.bsm_call_price(sigma) - self.market_price

    def secant_root(self, x0, x1, n_iter=100, tol=1e-6):
        a, b = self.D  # The interval for which we are searching for the roots
        
        # Check if initial guesses are within bounds
        if x0 < a or x0 > b or x1 < a or x1 > b:
            raise ValueError("Initial guesses must be within the interval [a, b]")

        for i in range(n_iter):
            f_x0 = self.f(x0)
            f_x1 = self.f(x1)
            
            # Check if the difference between f(x1) and f(x0) is too small to avoid division by zero
            if abs(f_x1 - f_x0) < tol:
                raise ZeroDivisionError(f"f(x0) and f(x1) too close at iteration {i}.")
            
            # Update step
            x_next = x1 - f_x1 * (x1 - x0) / (f_x1 - f_x0)
            
            # Make sure x_next stays within the interval [a, b]
            if x_next < a:
                x_next = a
            elif x_next > b:
                x_next = b
            
            # Check for convergence
            if abs(self.f(x_next)) < tol:
                return x_next
            
            # Update for the next iteration
            x0, x1 = x1, x_next  # Shift x0 to x1, and x1 to x_next

        raise ValueError("Secant method did not converge")

In [95]:
D = [0.01, 1.0]  # Volatility range
S = 40  # Underlying asset price
K = 35  # Strike price
r = 0.08  # Risk-free interest rate
T = 1  # Time to maturity
market_price = 10  # Market price of the call option

In [97]:
# Create an instance of the SecantMethod class
secant = SecantMethod(D, S, K, r, T, market_price)

# Use two initial guesses
x0 = 0.1
x1 = 0.2

# Find the implied volatility using the Secant Method
implied_vol = secant.secant_root(x0, x1)
print(f"Implied Volatility: {implied_vol}")

Implied Volatility: 0.37016162078834
